# 파이썬 병행성
- 병행성, 흐름제어 설명
- 이터레이터(Iterator)
- 제너리에터(Generator)
- `__iter__`, `__next__`
- 클래스 기반 제너레이터 구현

## 병행성 기본 - 이터레이터와 제너레이터

### 이터레이터(Iterator)

- 파이썬 제너레이너는 이터레이터를 만들어 낸다.

- 간단히 말해 제너레이터는 반복할 수 있는 이터레이터 오브젝트를 리턴한다.

- 파이썬 반복 가능한 타입

    - for, collections, text file, List, Dict, Set, Tuple, unpacking, *args

In [1]:
# 반복 가능한 이유? -> iter(x) 함수 호출
t = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

# for c in t:
#     print(c)

print(dir(t))

['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill']


- `__iter__`가 있으므로 반복이 가능하다.
- 내부적으로 `__iter__`를 받은 후 `__next__`를 호출한다.

In [2]:
# while 반복
w = iter(t) # 직접 iter 함수 호출

print(dir(w))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__length_hint__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__']


- `__iter__`도 있고 `__next__`도 있다.

In [3]:
print(next(w))
print(next(w))
print(next(w))

A
B
C


- 내부적으로 `iter`라는 함수를 호출하여 반복할 수 있다.

- 내부적으로 t가 `iter` 함수를 호출해서 iterable한 것을 받아 `next` 함수를 호출하여 다음 것을 받아온다.

In [4]:
while True:
    try:
        print(next(w))
    except StopIteration:
        break

D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z


- `while`문으로 `next`를 계속 호출하고 `StopIteration` 예외가 발생하면 `while`문 탈출한다.

- 위의 매커니즘이 `for`문에서 내부적으로 적용되는 것이다.

In [5]:
from collections import abc

# 반복형 확인
print(hasattr(t, '__iter__'))
print(isinstance(t, abc.Iterable)) # 상속을 받았는지 확인

True
True


- `hasattr`는 속성을 확인한다.

- `isinstance(object, class )`는 첫 번째 인자로 인스턴스, 두 번째 인자로 클래스 이름을 받는다. 입력으로 받은 인스턴스가 그 클래스의 인스턴스인지를 판단하여 참이면 True, 거짓이면 False를 돌려준다.

### 클래스 기반 제너레이터

In [6]:
# next 사용
class WordSplitter:
    def __init__(self, text):
        self._idx = 0
        self._text = text.split(' ')

    # 매직 메소드로 next 함수를 구현
    def __next__(self):
        # print('Called __next__')
        try:
            word = self._text[self._idx]
        except IndexError:
            raise StopIteration('Stopped Iteration.')
        self._idx += 1
        return word

    def __repr__(self):
        return 'WordSplit(%s)' % (self._text)

In [7]:
wi = WordSplitter('Do today what you could do tomorrow')

print(wi)
print(next(wi))
print(next(wi))
print(next(wi))
print(next(wi))
print(next(wi))
print(next(wi))
print(next(wi))
# print(next(wi)) # error

WordSplit(['Do', 'today', 'what', 'you', 'could', 'do', 'tomorrow'])
Do
today
what
you
could
do
tomorrow


- 매직 메소드로 `next` 함수를 구현하여 iterable하게 만들어준다.

- 클래스이지만 iterable 하게 되었다.

- 하지만 위의 `next` 패턴은 예외처리등 불편하다.

- 이걸 `generator` 패턴으로 바꾸면 한결 코드가 간결해진다.

### 제너레이터 패턴

**Generator 패턴**

1. 지능형 리스트, 딕셔너리, 집합 -> 데이터 양 증가 후 메모리 사용량 증가 -> 제네레이터 사용 권장
2. 단위 실행 가능한 `코루틴(Coroutine)` 구현과 연동
3. 작은 메모리 조각 사용

In [8]:
class WordSplitGenerator:
    def __init__(self, text):
        self._text = text.split(' ')

    # 매직 메소드로 __iter__ 함수 구현
    def __iter__(self):
        # print('Called __iter__')
        for word in self._text:
            yield word  # 제네레이터
        return

    def __repr__(self):
        return 'WordSplit(%s)' % (self._text)

In [9]:
wg = WordSplitGenerator('Do today what you could do tomorrow')

wt = iter(wg) # iter 함수 호출

print(wt)
print(next(wt))
print(next(wt))
print(next(wt))
print(next(wt))
print(next(wt))
print(next(wt))
print(next(wt))
# print(next(wt))

<generator object WordSplitGenerator.__iter__ at 0x000002D2C82062C8>
Do
today
what
you
could
do
tomorrow


- `yield` 키워드에 의해서 리턴과 예외처리를 알아서 처리해준다.

## 제너레이터(Generator)
- 제너레이터 실습
- Yield 실습
- Itertools 실습

### 제너레이터 실습
- 병행성(Concurrency) : 한 컴퓨터가 여러 일을 동시에 수행

    - 단일 프로그램 안에서 여러 일을 쉽게 해결
    - 코루틴
    - 쓰레드는 하나이지만 여러 일을 동시에 하는 것처럼 행동
    
    

- 병렬성(Parallelism) : 여러 컴퓨터가 여러 작업을 동시에 수행

    - 속도

In [10]:
# Generator Ex1
import itertools


def generator_ex1():
    print('Start')
    yield 'A Point. '
    print('continue')
    yield 'B Point. '
    print('End')


temp = iter(generator_ex1()) # iter 함수에 gernerator_ex1을 넣는다

print(next(temp))
print("-" * 30)

print(next(temp))

# print(next(temp)) # StopIteration

Start
A Point. 
------------------------------
continue
B Point. 


```python
print('Start')
yield 'A Point.'
```
- start 출력 후 A Point에서 멈춰 있는다. 다음 `next`로 호출할 때까지 `yield`까지 실행 된 것이다.

```python
print('continue')
yield 'B Point.'
```
- `next`를 다시 호출하면 continue를 출력하고 B Point에서 멈춰있는다.
- 또 다시 `next`로 호출하면 End가 출력된 후 `StopIteration` 에러가 발생한다.

이러한 과정이 여러 일을 동시에 수행해주는 메커니즘이다.

`yield`에 문자 출력 대신 네이버에서 크롤링, 다음에서 크롤링 같은 코드를 넣으면 네이버에서 크롤링하고 다 될 때까지 잠시 멈춰있다가, 다음의 할 일을 기억한 상태에서 다른 일을 하다 다시 `next`를 호출하면 그 다음의 일을 수행하게 하는 것이 가능해진다.

이런식으로 단일 프로그램에서 여러일을 동시에 처리할 수 있게 해준다.

In [11]:
# for문에서도 반복이 가능하다
for v in generator_ex1():
    print(v)

Start
A Point. 
continue
B Point. 
End


- for문에서 바로 호출도 가능하다.

In [12]:
# Generator Ex2
temp2 = [x * 3 for x in generator_ex1()] # list
temp3 = (x * 3 for x in generator_ex1()) # generator

print(temp2) # list
print("-" * 50)
print(temp3) # generator

Start
continue
End
['A Point. A Point. A Point. ', 'B Point. B Point. B Point. ']
--------------------------------------------------
<generator object <genexpr> at 0x000002D2C8235DC8>


- `yield`뒤에 있는 리턴 값들이 x에 들어가게 된다. `yield`가 `return` 역할을 한다고 볼 수 있다.

In [13]:
# list
for i in temp2:
    print(i)

print("-" * 50)

# generator
for i in temp3:
    print(i)

A Point. A Point. A Point. 
B Point. B Point. B Point. 
--------------------------------------------------
Start
A Point. A Point. A Point. 
continue
B Point. B Point. B Point. 
End


- temp2는 A point가 3번 B point가 3번씩 반복한다. `yield`에서 리턴된 값들이 리스트 안에 들어간다.

- temp3는 제너레이터이다. 그러므로 `for`문으로 호출하면 print문부터 차례대로 출력이 된다.
     - continue까지 실행되고 멈추고, 다시 호출되면 End까지 실행된다.

### 제너레이터 중요 함수

- filterfalse, accumulate, chain, product, product, groupby

In [14]:
# 데이터를 무한으로 만들고 싶을 때

import itertools

# 데이터를 무한으로 만들어준다
gen1 = itertools.count(1, 2.5)

print(next(gen1))
print(next(gen1))
print(next(gen1))
print(next(gen1))
# ... 무한

1
3.5
6.0
8.5


- 무한대로 2.5씩 증가하는 수를 `itertools.count`를 이용해 만든다.

In [15]:
# 조건
# takewhile(함수, itertools.count)
gen2 = itertools.takewhile(lambda n: n < 1000, itertools.count(1, 2.5))

for v in gen2:
    print(v)

1
3.5
6.0
8.5
11.0
13.5
16.0
18.5
21.0
23.5
26.0
28.5
31.0
33.5
36.0
38.5
41.0
43.5
46.0
48.5
51.0
53.5
56.0
58.5
61.0
63.5
66.0
68.5
71.0
73.5
76.0
78.5
81.0
83.5
86.0
88.5
91.0
93.5
96.0
98.5
101.0
103.5
106.0
108.5
111.0
113.5
116.0
118.5
121.0
123.5
126.0
128.5
131.0
133.5
136.0
138.5
141.0
143.5
146.0
148.5
151.0
153.5
156.0
158.5
161.0
163.5
166.0
168.5
171.0
173.5
176.0
178.5
181.0
183.5
186.0
188.5
191.0
193.5
196.0
198.5
201.0
203.5
206.0
208.5
211.0
213.5
216.0
218.5
221.0
223.5
226.0
228.5
231.0
233.5
236.0
238.5
241.0
243.5
246.0
248.5
251.0
253.5
256.0
258.5
261.0
263.5
266.0
268.5
271.0
273.5
276.0
278.5
281.0
283.5
286.0
288.5
291.0
293.5
296.0
298.5
301.0
303.5
306.0
308.5
311.0
313.5
316.0
318.5
321.0
323.5
326.0
328.5
331.0
333.5
336.0
338.5
341.0
343.5
346.0
348.5
351.0
353.5
356.0
358.5
361.0
363.5
366.0
368.5
371.0
373.5
376.0
378.5
381.0
383.5
386.0
388.5
391.0
393.5
396.0
398.5
401.0
403.5
406.0
408.5
411.0
413.5
416.0
418.5
421.0
423.5
426.0
428.5
431.0
433.5
43

- `takewhile`을 이용해서 1000 미만까지 수를 계속 만들어낸다.
- 이런식으로 `takewhile`과 `itertools.count`는 주로 같이 쓰인다.

In [16]:
# 필터 반대
gen3 = itertools.filterfalse(lambda n: n < 3, [1, 2, 3, 4, 5])

for v in gen3:
    print(v)

3
4
5


- `filterfalse`는 `filter`의 반대 역할을 한다.

In [17]:
# 누적 합계
gen4 = itertools.accumulate([x for x in range(1, 101)])

for v in gen4:
    print(v)

1
3
6
10
15
21
28
36
45
55
66
78
91
105
120
136
153
171
190
210
231
253
276
300
325
351
378
406
435
465
496
528
561
595
630
666
703
741
780
820
861
903
946
990
1035
1081
1128
1176
1225
1275
1326
1378
1431
1485
1540
1596
1653
1711
1770
1830
1891
1953
2016
2080
2145
2211
2278
2346
2415
2485
2556
2628
2701
2775
2850
2926
3003
3081
3160
3240
3321
3403
3486
3570
3655
3741
3828
3916
4005
4095
4186
4278
4371
4465
4560
4656
4753
4851
4950
5050


- `itertools.accumulate`로 누적 합계를 출력한다.

In [18]:
# 연결1
gen5 = itertools.chain('ABCDE', range(1, 11, 2))

print(list(gen5))

['A', 'B', 'C', 'D', 'E', 1, 3, 5, 7, 9]


- `chain`을 이용해 iterable한 두 개를 이어서 하나로 만든다

In [19]:
# 연결2
gen6 = itertools.chain(enumerate('ABCDE'))

print(list(gen6))

[(0, 'A'), (1, 'B'), (2, 'C'), (3, 'D'), (4, 'E')]


- `enumerate`가 인덱스 번호를 붙여주므로 위와 같이 쌍으로 튜플형 리스트가 나온다.

In [20]:
# 개별
gen7 = itertools.product('ABCDE')

print(list(gen7))

[('A',), ('B',), ('C',), ('D',), ('E',)]


- `itertools.product`를 사용하면 개별로 분리해서 튜플 리스트 형태로 만들어준다.

In [21]:
# 연산(경우의 수)
gen8 = itertools.product('ABCDE', repeat=2)

print(list(gen8))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('B', 'A'), ('B', 'B'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'A'), ('C', 'B'), ('C', 'C'), ('C', 'D'), ('C', 'E'), ('D', 'A'), ('D', 'B'), ('D', 'C'), ('D', 'D'), ('D', 'E'), ('E', 'A'), ('E', 'B'), ('E', 'C'), ('E', 'D'), ('E', 'E')]


- `itertools.product`에 `repeat` 옵션을 붙여줘 모든 경우의 수를 따져서 순서쌍 맺어준다.

In [22]:
# 그룹화
gen9 = itertools.groupby('AAABBCCCCDDEEE')

# print(list(gen9))

for chr, group in gen9:
    print(chr, ' : ', list(group))

A  :  ['A', 'A', 'A']
B  :  ['B', 'B']
C  :  ['C', 'C', 'C', 'C']
D  :  ['D', 'D']
E  :  ['E', 'E', 'E']


- `itertools.groupby`로 그룹화를 해준다.

## 코루틴(Coroutine), Yield

- 병행성(Concurrency) : 한 컴퓨터가 여러 일을 동시에 수행 -> 단일 프로그램안에서 여러 일을 쉽게 해결
- 병렬성(Parallelism) : 여러 컴퓨터가 여러 작업을 동시에 수행 -> 속도


**쓰레드** 
- OS에서 직접 관리, 동시에 각자의 일이나 한가지 일 수행, 
- CPU 코어에서 실시간 혹은 시분할로 나눠서 비동기 작업 -> `멀티 쓰레드`

**코루틴**
- `단일(싱글) 스레드` 안에서 `메인루틴`과 `서브루틴`이 상호작용을 하면서 작업 
- 스택을 기반으로 동작하는 `비동기 작업`
- 리소스가 부족한 단일 스레드 환경 아래에서 하나의 작업을 순차적으로 하면서 블럭되지 않고 작업을 한다.
- 코루틴을 제어할 때는 `상태`를 저장하고 `양방향 전송`한다.
- 서브루틴은 `메인 루틴`에서 호출하면 `서브 루틴`에서 수행한다.(흐름 제어)
- 루틴 실행 중 중지를 할 수 있으므로 `동시성 프로그래밍`이라고도 한다.
- 코루틴의 장점은 쓰레드에 비해 오버헤드가 감소한다는 장점이 있다.
    - 멀티 스레드는 교착 상태 발생 가능성, 컨텍스트 스위칭 비용 발생, 자원 소비 가능성 증가한다.

### yield, send
- `yield`와 `send` 키워드를 통해서 메인과 서브루틴이 상호작용을 할 것이다.
- 제너레이터(값을 반복적으로 생성)에서 파생된 것이 코루틴이다.

In [23]:
# 코루틴 Ex1
from inspect import getgeneratorstate


def coroutine1():
    print('>>> coroutine started.')
    i = yield  # i에 yield 할당
    print('>>> coroutine received : {}'.format(i))

- i에 `yield`를 할당한다.
    - 함수를 보지 말고 안의 제너레이터를 봐야 한다.
    - 함수와 제너레이터, 코루틴은 모두 `def`로 만들기 떄문에 구분을 잘해야 한다.
- `yield`라는 키워드가 들어가면 제너레이터이다. 그래서 제너레이터 기반 코루틴이라고 설명하기도 한다.

In [24]:
# 메인 루틴

cr1 = coroutine1() # 제네레이터 선언

print(cr1, type(cr1))

<generator object coroutine1 at 0x000002D2C8206348> <class 'generator'>


- 함수처럼 제너레이터를 선언해준다.
- 타입은 `genrerator`이다.

In [25]:
# yield 지점 까지 서브루틴 수행
next(cr1)

>>> coroutine started.


- `next`를 이용해 서브루틴을 한 번 수행하면 coroutine started까지 출력된다.

In [26]:
# 기본 전달 값 None
# next(cr1) # StopItertation

In [27]:
# 값 전송

# 메인 루틴에서 서브루틴으로 100 보낸다
# cr1.send(100) # StopIteration

- `send` 명령어를 통해서 메인루틴과 서브루틴이데이터를 주고 받을 수 있다.

- `send`는 `next` 기능도 포함하고 있다.

- `send`에 아무것도 넣지 않으면 기본값으로 `None`이 전달된다.

In [28]:
# 잘못된 사용
cr2 = coroutine1()

# next(cr1) # next 없이 바로 send를 실행하면 예외 발생

# cr2.send(100) # 예외 발생

- `next` 없이 바로 `send`를 실행하면 예외 발생한다.

- `yield`에서 멈춘 다음에 `send`를 보내야 한다.

### 코루틴 상태값
- `GEN_CREATED` : 처음 대기 상태
- `GEN_RUNNING` : 실행 상태
- `GEN_SUSPENDED` : yield 대기 상태
- `GEN_CLOSED` : 실행 완료 상태

In [29]:
# 매개변수가 있는 코루틴
from inspect import getgeneratorstate

def coroutine2(x):
    print('>>> coroutine started : x = {}'.format(x)) # 받았는지 확인하기 위해 x값 출력
    y = yield x  # x를 서브루틴에서 메인루틴으로 전달
    print('>>> coroutine received : y = {}'.format(y))
    z = yield x + y
    print('>>> coroutine received : z = {}'.format(z))

In [30]:
cr3 = coroutine2(100) # 시작시에 메인 루틴에서 서브루틴으로 100을 넣고 시작

# print(getgeneratorstate(cr3))  # 코루틴 상태 출력

print(next(cr3)) # yield x 리턴, y 값 받을 대기 상태
print(getgeneratorstate(cr3))  # 코루틴 상태 출력

print(cr3.send(200)) # y 값으로 200을 보낸다, z를 받기 위한 상태가 된다
print(getgeneratorstate(cr3))  # 코루틴 상태 출력

# print(cr3.send(300))  # 예외 StopIteration

>>> coroutine started : x = 100
100
GEN_SUSPENDED
>>> coroutine received : y = 200
300
GEN_SUSPENDED


**1. 제너레이터 생성**
```python
cr3 = coroutine2(100) # 100을 넣고 시작
```

- 시작시에 메인 루틴에서 서브루틴으로 `x` 값 100을 넣고 시작한다. (`GEN_CREATED`)

    ```python
    print(getgeneratorstate(cr3))  # 코루틴 상태 출력
    ```
    - `getgeneratorstate`으로 코루틴의 상태를 출력한다.

**2. `next(cr3)`**
```python
next(cr3) # yield x 리턴, y 값 받을 대기 상태
```

- `yield x`가 반환(return)되어 메인 루틴으로 보내졌다.
    ```python
    y = yield x  # x를 서브루틴에서 메인루틴으로 전달
    ```
- 그리고 `y`값을 받을 대기 상태가 된다. (`GEN_SUSPENDED`)

**3. `send(200)`**

```python
cr3.send(200) # y 값으로 200을 보낸다, z를 받기 위한 상태가 된다
```


- `y`값으로 200을 메인 루틴에서 서브루틴으로 보낸다. 
- `z`를 받기 위한 상태가 된다. (`GEN_SUSPENDED`)
- 이 때 `yield x + y`가 서브루틴에서 메인 루틴으로 전달(리턴)한다.
    ```python
    z = yield x + y
    ```
    - 양방향 전송이다.
    - 오른쪽에 있으면 메인루틴에 보내는거고, 왼쪽에 있으면 입력을 받는거다.
    
    
**4. `send(300)`**

- `z` 값으로 메인루틴에서 서브루틴으로 300을 `send`한다.
- `StopIteration` 예외가 발생한다.



- 이런식으로 동시성 개발을 할 수 있다. 제너레이터를 기반으로 시작된다. 제너레이터는 클로저의 상태(free variable)를 저장한다.
- 파이썬 3.5 이상부터 def → `async`, yield → `await` 키워드로 바꿀 수 있다.

In [31]:
# 코루틴 Ex3
# StopIteration 자동 처리(3.5 -> await)
# 중첩 코루틴 처리


def generator1():
    for x in 'AB':
        yield x
    for y in range(1, 4):
        yield y


t1 = generator1()

print(next(t1))
print(next(t1))
print(next(t1))
print(next(t1))
print(next(t1))
# print(next(t1)) # StopIteration

t2 = generator1()

print(list(t2)) # 리스트로 형변환

A
B
1
2
3
['A', 'B', 1, 2, 3]


In [32]:
def generator2():
    yield from 'AB'
    yield from range(1, 4)


t3 = generator2()

print(next(t3))
print(next(t3))
print(next(t3))
print(next(t3))
print(next(t3))
# print(next(t3))

A
B
1
2
3


- 위의 코드를 `yield from` 키워드로 대체할 수 있다.
- `yield from`으로 반복 가능한 객체의 요소를 하나씩 바깥으로 보낸다.

## 병렬성 1 - Futures

- 비동기 작업 처리
- 파이썬 GIL 설명
- 동시성 처리 실습 예제
- Process, Thread 예제

### 동시성

#### **비동기 asynchronous**

- `병렬성` 혹은 `동시성`은 최대한 자원을 활용해서 여러개의 작업을 동시에 처리할 수 있게 해준다.

- `비동기(asynchronous)`는 A → B → END 의 프로세스에서 A의 작업이 시작되면 끝날 때까지 기다리지 않고 B의 작업을 시작하는 것이다.
    - 반면 `동기(synchronous)`는 A의 일이 끝나야만이 B의 일을 시작할 수 있다.
    - 지연시간(block) CPU 및 리소스 낭비 방지 
    - (File)Network I/O 관련 작업등에서는 동시성 활용이 권장된다
    - 비동기 작업과 적합한 프로그램일 경우 압도적으로 성능이 향상된다.


#### **`futures`**
- `futures` : 비동기 실행을 위한 API를 고수준으로 작성하여 사용하기 쉽도록 개선한 것이 `futures`이다.
    - `threading`, `multiprocessing`을 wrapping 하였다.
    
    
    
#### **`concurrent.Futures`**

1. 멀티스레딩/멀티프로세싱 API 통일 -> 매우 사용하기 쉬움
2. 실행중이 작업 취소, 완료 여부 체크, 타임아웃 옵션, 콜백추가, 동기화 코드 매우 쉽게 작성 -> Promise 개념

#### **GIL**
- `GIL(Global Interface Lock)`은 여러개의 작업을 실행할 떄 전체가 락이 걸리는 현상을 말한다. 파이썬에만 있는 독특한 특징이다.
- 두 개 이상의 스레드가 동시에 실행 될 때 하나의 자원을 엑세스 하는 경우 문제점을 방지하기 위해 GIL이 실행된다. 즉, 리소스 전체가 락이 걸린다. 게다가 문맥 교환 비용이 든다.
- 서로 일한 것을 합치는 과정을 `문맥 교환(Context Switch)`이라 한다.
- 그래서 여러개의 쓰레드로 작업하면 오히려 하나만 쓸 때보다 느릴 때가 있다.


- 그래서 GIL을 우회하기 위해 멀티 프로세싱을 사용하고나 CPython을 사용한다.
- 결론은 파이썬에 GIL이라는 특징이 있기 떄문에 무조건 쓰레드를 많이 사용하는 것이 좋은 것이 아니다.

### `concurrent.futures` 사용법1 - Thread 사용

- [프로세스와 쓰레드의 차이](https://gmlwjd9405.github.io/2018/09/14/process-vs-thread.html)


- [concurrent.futures — 병렬 작업 실행하기](https://docs.python.org/ko/3.7/library/concurrent.futures.html)
- `concurrent.futures` 모듈은 비동기적으로 콜러블을 실행하는 고수준 인터페이스를 제공합니다.

- 비동기 실행은 (`ThreadPoolExecutor`를 사용해서) 스레드나 (`ProcessPoolExecutor`를 사용해서) 별도의 프로세스로 수행 할 수 있습니다. 둘 다 추상 Executor 클래스로 정의된 것과 같은 인터페이스를 구현합니다.

In [3]:
import os
import time # 시간 측정
from concurrent import futures # futures

- `futures` 안에 `threading`, `multiprocessing` 관련 코드가 포함되어 있다.

In [4]:
# 동시에 4개의 worker에게 일을 시켜서 각각 1부터의 합을 구한다.

WORK_LIST = [100000, 1000000, 10000000, 10000000]

In [5]:
# 동시성 합계 계산 메인함수 - 누적 합계 함수(제네레이터)


def sum_generator(n):
    return sum(n for n in range(1, n + 1))

- 동시성이므로 위의 함수(`sum_generator`)가 한 번에 4개가 실행된다.

In [6]:
# 진입점
def main():
    # Worker Count
    # worker 수를 10개 혹은 리스트의 원소 갯수 중 최소값으로 지정한다.
    worker = min(10, len(WORK_LIST))

    start_tm = time.time()  # 시작 시간

    # ThreadPoolExecutor
    with futures.ThreadPoolExecutor() as excutor:
        # map -> 작업 순서 유지, 즉시 실행
        result = excutor.map(sum_generator, WORK_LIST)

    end_tm = time.time() - start_tm  # 종료 시간
    
    msg = '\n Result -> {} Time : {:.2f}s'  # 출력 포맷
    print(msg.format(list(result), end_tm))  # 최종 결과


#  시작점 명시적 작성
if __name__ == '__main__':
    main()


 Result -> [5000050000, 500000500000, 50000005000000, 50000005000000] Time : 1.45s


```python
worker = min(10, len(WORK_LIST))
```
- worker의 숫자를 10개 혹은 리스트의 원소 갯수 중 최소값으로 지정한다.

**`class concurrent.futures.Executor`**

- 비동기적으로 호출을 실행하는 메서드를 제공하는 추상 클래스입니다. 직접 사용해서는 안 되며, 구체적인 하위 클래스를 통해 사용해야 합니다.

**`ThreadPoolExecutor`**

- `ThreadPoolExecutor` 는 스레드 풀을 사용하여 호출을 비동기적으로 실행하는 `Executor` 서브 클래스입니다.
```python
# ThreadPoolExecutor
with futures.ThreadPoolExecutor() as excutor:
    # map -> 작업 순서 유지, 즉시 실행
    result = excutor.map(sum_generator, WORK_LIST)
```
- 4가지 연산을 동시에 한다.

### `concurrent.futures` 사용법2 - Process 사용

In [7]:
# 진입점
def main():
    # Worker Count
    # worker 수를 10개 혹은 리스트의 원소 갯수 중 최소값으로 지정한다.
    worker = min(10, len(WORK_LIST))

    start_tm = time.time()  # 시작 시간

    # ProcessPoolExecutor
    with futures.ProcessPoolExecutor() as excutor:
        # map -> 작업 순서 유지, 즉시 실행
        result = excutor.map(sum_generator, WORK_LIST)

    end_tm = time.time() - start_tm  # 종료 시간
    msg = '\n Result -> {} Time : {:.2f}s'  # 출력 포맷
    print(msg.format(list(result), end_tm))  # 최종 결과
    
#  시작점 명시적 작성
if __name__ == '__main__':
    main()

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

**`ProcessPoolExecutor`**
```python
# ProcessPoolExecutor
with futures.ProcessPoolExecutor() as excutor:
    # map -> 작업 순서 유지, 즉시 실행
    result = excutor.map(sum_generator, WORK_LIST)
```
- `ProcessPoolExecutor` 클래스는 프로세스 풀을 사용하여 호출을 비동기적으로 실행하는 `Executor` 서브 클래스입니다. 
- `ProcessPoolExecutor` 는 `multiprocessing` 모듈을 사용합니다. 전역 인터프리터 록 을 피할 수 있도록 하지만, 오직 피클 가능한 객체만 실행되고 반환될 수 있음을 의미합니다.


- `__main__` 모듈은 작업자 서브 프로세스가 임포트 할 수 있어야 합니다. 즉, `ProcessPoolExecutor` 는 대화형 인터프리터에서 작동하지 않습니다.


- 단순계산에서는 프로세스가 쓰레드를 사용하는 것보다 조금 더 빠르다.

### Thread and Process

- 프로세스는 식당, 쓰레드는 종업원에 비유할 수 있다.


- 싱글 쓰레드는 식당에 종업원이 한 명 있는 경우이다.
    - 먼저 온 손님의 주문을 받고, 주방에서 조리하는 사이에 다른 손님의 주문을 받을 수 있다.
- 멀티 쓰레드는 식당에 종업원이 여러명 있는 경우이다.
    - 각 손님에게 종업원 한명씩 할당에 서빙할 수 있다. 
    - 하지만 손님이 없는 경우 자원의 낭비가 된다.
    
    
- 프로세스는 식당에 비유할 수 있다. 
    - 하나의 식당(프로세스)는 하나의 프로그램이다.
- 멀티 프로세스로 작업을 하면 각기 다른 식당(한식, 중식, 일식)이 동시에 일을 하여 나중에 음식을 합쳐 코스 요리로 낼 수 있게 된다.

**Reference**
- [프로세스와 쓰레드의 차이](https://gmlwjd9405.github.io/2018/09/14/process-vs-thread.html)

#### 프로그램(Program) 이란

- 사전적 의미
    - “어떤 작업을 위해 실행할 수 있는 파일”

#### 프로세스(Process) 란


<img src="https://gmlwjd9405.github.io/images/os-process-and-thread/process.png" alt="Process" style="width: 500px;"/>

- 사전적 의미
    - “컴퓨터에서 연속적으로 실행되고 있는 컴퓨터 프로그램”
    - 메모리에 올라와 실행되고 있는 프로그램의 인스턴스(독립적인 개체)
    - 운영체제로부터 시스템 자원을 할당받는 작업의 단위
    - 즉, 동적인 개념으로는 실행된 프로그램을 의미한다.
        

- 참고: 할당받는 시스템 자원의 예
    - CPU 시간
    - 운영되기 위해 필요한 주소 공간
    - Code, Data, Stack, Heap의 구조로 되어 있는 독립된 메모리 영역


- 특징
    - 프로세스는 각각 독립된 메모리 영역(Code, Data, Stack, Heap의 구조)을 할당받는다.
    - 기본적으로 프로세스당 최소 1개의 스레드(메인 스레드)를 가지고 있다.
    - 각 프로세스는 별도의 주소 공간에서 실행되며, 한 프로세스는 다른 프로세스의 변수나 자료구조에 접근할 수 없다.
    - 한 프로세스가 다른 프로세스의 자원에 접근하려면 프로세스 간의 통신(IPC, inter-process communication)을 사용해야 한다.
        - Ex. 파이프, 파일, 소켓 등을 이용한 통신 방법 이용

        
#### 스레드(Thread) 란

<img src="https://gmlwjd9405.github.io/images/os-process-and-thread/thread.png" alt="Thread" style="width: 500px;"/>

- 사전적 의미
    - “프로세스 내에서 실행되는 여러 흐름의 단위”
    - 프로세스의 특정한 수행 경로
    - 프로세스가 할당받은 자원을 이용하는 실행의 단위


- 특징
    - 스레드는 프로세스 내에서 각각 Stack만 따로 할당받고 Code, Data, Heap 영역은 공유한다.
    - 스레드는 한 프로세스 내에서 동작되는 여러 실행의 흐름으로, 프로세스 내의 주소 공간이나 자원들(힙 공간 등)을 같은 프로세스 내에 스레드끼리 공유하면서 실행된다.
    - 같은 프로세스 안에 있는 여러 스레드들은 같은 힙 공간을 공유한다. 반면에 프로세스는 다른 프로세스의 메모리에 직접 접근할 수 없다.
    - 각각의 스레드는 별도의 레지스터와 스택을 갖고 있지만, 힙 메모리는 서로 읽고 쓸 수 있다.
    - 한 스레드가 프로세스 자원을 변경하면, 다른 이웃 스레드(sibling thread)도 그 변경 결과를 즉시 볼 수 있다.


#### 멀티 프로세스와 멀티 스레드의 차이
##### 멀티 프로세스

- 멀티 프로세싱이란
    - 하나의 응용프로그램을 여러 개의 프로세스로 구성하여 각 프로세스가 하나의 작업(태스크)을 처리하도록 하는 것이다.


- 장점
    - 여러 개의 자식 프로세스 중 하나에 문제가 발생하면 그 자식 프로세스만 죽는 것 이상으로 다른 영향이 확산되지 않는다.


- 단점
    - Context Switching에서의 오버헤드
        - Context Switching 과정에서 캐쉬 메모리 초기화 등 무거운 작업이 진행되고 많은 시간이 소모되는 등의 오버헤드가 발생하게 된다.
        - 프로세스는 각각의 독립된 메모리 영역을 할당받았기 때문에 프로세스 사이에서 공유하는 메모리가 없어, Context Switching가 발생하면 캐쉬에 있는 모든 데이터를 모두 리셋하고 다시 캐쉬 정보를 불러와야 한다.

    - 프로세스 사이의 어렵고 복잡한 통신 기법(IPC)
        - 프로세스는 각각의 독립된 메모리 영역을 할당받았기 때문에 하나의 프로그램에 속하는 프로세스들 사이의 변수를 공유할 수 없다.



- 참고 Context Switching란?
    - CPU에서 여러 프로세스를 돌아가면서 작업을 처리하는 데 이 과정을 Context Switching라 한다.
    - 구체적으로, 동작 중인 프로세스가 대기를 하면서 해당 프로세스의 상태(Context)를 보관하고, 대기하고 있던 다음 순서의 프로세스가 동작하면서 이전에 보관했던 프로세스의 상태를 복구하는 작업을 말한다. 


##### 멀티 스레드

- 멀티 스레딩이란
    - 하나의 응용프로그램을 여러 개의 스레드로 구성하고 각 스레드로 하여금 하나의 작업을 처리하도록 하는 것이다.
    - 윈도우, 리눅스 등 많은 운영체제들이 멀티 프로세싱을 지원하고 있지만 멀티 스레딩을 기본으로 하고 있다.
    - 웹 서버는 대표적인 멀티 스레드 응용 프로그램이다.


- 장점
    - 시스템 자원 소모 감소 (자원의 효율성 증대)
        - 프로세스를 생성하여 자원을 할당하는 시스템 콜이 줄어들어 자원을 효율적으로 관리할 수 있다.
    - 시스템 처리량 증가 (처리 비용 감소)
        - 스레드 간 데이터를 주고 받는 것이 간단해지고 시스템 자원 소모가 줄어들게 된다.
        - 스레드 사이의 작업량이 작아 Context Switching이 빠르다.
    - 간단한 통신 방법으로 인한 프로그램 응답 시간 단축
        - 스레드는 프로세스 내의 Stack 영역을 제외한 모든 메모리를 공유하기 때문에 통신의 부담이 적다.


- 단점
    - 주의 깊은 설계가 필요하다.
    - 디버깅이 까다롭다.
    - 단일 프로세스 시스템의 경우 효과를 기대하기 어렵다.
    - 다른 프로세스에서 스레드를 제어할 수 없다. (즉, 프로세스 밖에서 스레드 각각을 제어할 수 없다.)
    - 멀티 스레드의 경우 자원 공유의 문제가 발생한다. (동기화 문제)
    - 하나의 스레드에 문제가 발생하면 전체 프로세스가 영향을 받는다.


#### 멀티 프로세스 대신 멀티 스레드를 사용하는 이유?

<img src="https://gmlwjd9405.github.io/images/os-process-and-thread/multi-thread.png" alt="MultiThread" style="width: 700px;"/>

- 멀티 프로세스 대신 멀티 스레드를 사용하는 것의 의미?
    - 쉽게 설명하면, 프로그램을 여러 개 키는 것보다 하나의 프로그램 안에서 여러 작업을 해결하는 것이다.


    - 여러 프로세스(멀티 프로세스)로 할 수 있는 작업들을 하나의 프로세스에서 여러 스레드로 나눠가면서 하는 이유?
        - 자원의 효율성 증대
            - 멀티 프로세스로 실행되는 작업을 멀티 스레드로 실행할 경우, 프로세스를 생성하여 자원을 할당하는 시스템 콜이 줄어들어 자원을 효율적으로 관리할 수 있다.
        - –> 프로세스 간의 Context Switching시 단순히 CPU 레지스터 교체 뿐만 아니라 RAM과 CPU 사이의 캐쉬 메모리에 대한 데이터까지 초기화되므로 오버헤드가 크기 때문
        - 스레드는 프로세스 내의 메모리를 공유하기 때문에 독립적인 프로세스와 달리 스레드 간 데이터를 주고 받는 것이 간단해지고 시스템 자원 소모가 줄어들게 된다.
    - 처리 비용 감소 및 응답 시간 단축
        - 또한 프로세스 간의 통신(IPC)보다 스레드 간의 통신의 비용이 적으므로 작업들 간의 통신의 부담이 줄어든다.
    - –> 스레드는 Stack 영역을 제외한 모든 메모리를 공유하기 때문
    - 프로세스 간의 전환 속도보다 스레드 간의 전환 속도가 빠르다.
    - –> Context Switching시 스레드는 Stack 영역만 처리하기 때문

- 주의할 점!
    - 동기화 문제
    - 스레드 간의 자원 공유는 전역 변수(데이터 세그먼트)를 이용하므로 함께 상용할 때 충돌이 발생할 수 있다.


## 병렬성 2 - Futures
- Futures wait 예제
- Futures as_completed
- 동시 실행 결과 출력
- 동시성 처리 응용 예제 설명


- 각각의 작업이 같은 시간이 걸린다는 보장이 없고 더구나 성공한다는 보장도 없다. 
- 이러한 세세한 작업 흐름을 컨트롤 해야 진정한 동시성 프로그래밍이라 할 수 있다.


- GIL : 두 개 이상의 스레드가 동시에 실행 될 때 하나의 자원을 엑세스 하는 경우 -> 문제점을 방지하기 위해
    - GIL 실행 , 리소스 전체에 락이 걸린다. -> Context Switch(문맥 교환)
    - 대체: 멀티프로세싱 사용, CPython

### `concurrent.futures.wait`

- [concurrent.futures — 병렬 작업 실행하기](https://docs.python.org/ko/3/library/concurrent.futures.html)


`concurrent.futures.wait(fs, timeout=None, return_when=ALL_COMPLETED)`

- `fs` 로 주어진 여러 (서로 다른 `Executor` 인스턴스가 만든 것들도 가능합니다) `Future` 인스턴스들이 완료할 때까지 기다립니다. 
- 집합들의 이름있는 2-튜플을 돌려줍니다. 
- `done` 이라는 이름의 첫 번째 집합은 대기가 끝나기 전에 완료된 (끝났거나 취소된) 퓨처를 담고 있습니다. 
- `not_done` 이라는 이름의 두 번째 집합은 완료되지 않은 (계류 중이거나 실행 중인) 퓨처를 담고 있습니다.

In [38]:
import os
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, wait, as_completed

WORK_LIST = [
    100000000,
    100000,
    10000,
    1000000,
]


# 동시성 합계 계산 메인 함수
# 누적 합계 함수(제레네이터)
def sum_generator(n):
    return sum(n for n in range(1, n+1))

In [39]:
# wait
def main():
    worker = min(10, len(WORK_LIST))  # Worker 카운트
    start_tm = time.time()  # 시작 시간
    futures_list = []  # future

    # ProcessPoolExecutor
    with ThreadPoolExecutor() as excutor:
        for work in WORK_LIST:
            # future 반환(아직 실행되지 않는다. 미래에 할일을 반환할 뿐이다)
            future = excutor.submit(sum_generator, work)

            # 스케쥴링(4개의 일을 담는다)
            futures_list.append(future)

            # 스케쥴링 확인
            print('Scheduled for {} : {}'.format(work, future))

        print()

        # wait 결과 출력
        result = wait(futures_list, timeout=7)  # 7초까지 기다려주고 7초안에 끝내지 못하면 중단

        print('Completed Tasks : ' + str(result.done))  # 성공 task 출력
        print()
        print('Pending ones after waiting for 7 seconds : ' +
              str(result.not_done))  # 실패 task 출력
        print()
        print('Result : ', end="")
        print([future.result() for future in result.done])  # 결과 출력

    end_tm = time.time() - start_tm  # 종료 시간
    msg = '\n Time : {:.2f}s'  # 출력 포맷
    print(msg.format(end_tm))  # 최종 결과 출력


# 실행
if __name__ == '__main__':
    main()

Scheduled for 100000000 : <Future at 0x2d2c825d5c8 state=running>
Scheduled for 100000 : <Future at 0x2d2c9272648 state=running>
Scheduled for 10000 : <Future at 0x2d2c92727c8 state=finished returned int>
Scheduled for 1000000 : <Future at 0x2d2c824d648 state=running>

Completed Tasks : {<Future at 0x2d2c92727c8 state=finished returned int>, <Future at 0x2d2c824d648 state=finished returned int>, <Future at 0x2d2c9272648 state=finished returned int>}

Pending ones after waiting for 7 seconds : {<Future at 0x2d2c825d5c8 state=running>}

Result : [50005000, 500000500000, 5000050000]

 Time : 7.40s


- 계산하는데 7초가 넘은 것은 실패로 간주한다.
- future 하나는 실패하였다.


- `Excutor`의 `submit` 메소드
    - `submit(fn, *args, **kwargs)`
    - 콜러블 fn 이 `fn(*args **kwargs)` 처럼 실행되도록 예약하고, 콜러블 객체의 실행을 나타내는 Future 객체를 반환합니다.
    
- `wait`
    ```python
    # wait 결과 출력
    result = wait(futures_list, timeout=7)  # 7초까지 기다려주고 7초안에 끝내지 못하면 중단
    ```
    - `wait`에 `timeout` 옵션을 넣어 7초까지 기다려주고 7초 안에 끝내지 못하면 중단한다.
    - `timeout` 은 반환하기 전에 대기 할 최대 시간(초)을 제어하는 데 사용할 수 있습니다. 

### `concurrent.futures.as_completed`


`concurrent.futures.as_completed(fs, timeout=None)`

- `fs` 로 주어진 여러 (서로 다른 `Executor` 인스턴스가 만든 것들도 가능합니다) 퓨처들이 완료되는 대로 (끝났거나 취소된 퓨처) 일드 하는 `Future` 인스턴스의 이터레이터를 반환합니다. 
- `fs` 에 중복된 퓨처가 들어있으면 한 번만 반환됩니다. 
- `as_completed()` 가 호출되기 전에 완료한 모든 퓨처들이 먼저 일드 됩니다. 
- 반환된 이터레이터는, `__next__()` 가 호출되고, `as_completed()` 호출 시점으로부터 `timeout` 초 후에 결과를 얻을 수 없는 경우 `concurrent.futures.TimeoutError` 를 발생시킵니다. `timeout` 은 `int` 또는 `float`가 될 수 있습니다. `timeout` 이 지정되지 않았거나 None 인 경우, 대기 시간에는 제한이 없습니다.


In [40]:
# as_completed
def main():
    worker = min(10, len(WORK_LIST))  # Worker 카운트
    start_tm = time.time()  # 시작 시간
    futures_list = []  # future

    # ProcessPoolExecutor
    with ThreadPoolExecutor() as excutor:
        for work in WORK_LIST:
            # future 반환(아직 실행되지 않는다. 미래에 할일을 반환할 뿐이다)
            future = excutor.submit(sum_generator, work)

            # 스케쥴링(4개의 일을 담는다)
            futures_list.append(future)

            # 스케쥴링 확인
            print('Scheduled for {} : {}'.format(work, future))

        print()

        # as_completed 결과 출력
        for future in as_completed(futures_list):
            result = future.result()  # 결과
            done = future.done()  # 끝났는지
            cancelled = future.cancelled  # 취소 됐는지

            # future 결과 확인
            print('Future Result : {}, Done : {}'.format(result, done))  # 완료
            print('Future Cancelled : {}'.format(cancelled))  # 취소
            print()

    end_tm = time.time() - start_tm  # 종료 시간
    msg = '\n Time : {:.2f}s'  # 출력 포맷
    print(msg.format(end_tm))  # 최종 결과 출력


# 실행
if __name__ == '__main__':
    main()

Scheduled for 100000000 : <Future at 0x2d2c9278188 state=running>
Scheduled for 100000 : <Future at 0x2d2c9272e08 state=finished returned int>
Scheduled for 10000 : <Future at 0x2d2c9278c08 state=finished returned int>
Scheduled for 1000000 : <Future at 0x2d2c9275088 state=running>

Future Result : 50005000, Done : True
Future Cancelled : <bound method Future.cancelled of <Future at 0x2d2c9278c08 state=finished returned int>>

Future Result : 5000050000, Done : True
Future Cancelled : <bound method Future.cancelled of <Future at 0x2d2c9272e08 state=finished returned int>>

Future Result : 500000500000, Done : True
Future Cancelled : <bound method Future.cancelled of <Future at 0x2d2c9275088 state=finished returned int>>

Future Result : 5000000050000000, Done : True
Future Cancelled : <bound method Future.cancelled of <Future at 0x2d2c9278188 state=finished returned int>>


 Time : 7.60s


- `as_completed`는 먼저 끝나는 것이 먼저 반환된다.
- 작업의 양이 작고 금방금방 처리한다면 `wait`으로 처리하고, 먼저 끝나는대로 작업을 해야할 때면 `as_completed`를 사용한다.